In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import re 

%matplotlib inline

In [2]:
data_location = 'Data/properatti.csv'
data = pd.read_csv(data_location)

In [3]:
data_final = data[['property_type','place_name','place_with_parent_names','state_name','price','currency','price_aprox_usd',
                         'surface_total_in_m2','surface_covered_in_m2','price_per_m2','rooms','description','title']].copy()

In [4]:
#Buscamos aquellas propiedades sin Rooms.
mask_no_rooms = data_final["rooms"].isnull()
data_final.loc[mask_no_rooms,["rooms","description","title"]]

,rooms,description,title
0,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
...,...,...,...
121215,NaN,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes
121216,NaN,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...
121217,NaN,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121218,NaN,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...


In [5]:
#casteo columna description y title como string
data_final["description"] = data_final["description"].astype(str)
data_final["title"] = data_final["title"].astype(str)

In [6]:
#la cantidad de ambientes en description y title puede estar tanto en numeros como en numeros escritos en letras.
#Es por eso que buscamos ambos patrones en description y title

In [7]:
#Creo una funcion para la busqueda de patrones y me agregue los regex encontrados en una nueva columna al data frame
def buscador_patrones(patron, columna_de_trabajo, nombre_nueva_columna):
    patron_regex = re.compile(patron, re.IGNORECASE)
    matcheos = data_final[columna_de_trabajo].apply(lambda x:x if x is np.NaN else patron_regex.search(x).group("numero") if patron_regex.search(x)!=None else np.NaN)
    data_final.insert(13, nombre_nueva_columna, matcheos)

In [8]:
#Creamos patrones para buscar la cantidad explicita de ambientes en description y title.
description_ambientes_numero = "(?P<numero>\d\d?) ?(ambientes|ambiente|amb)"
description_ambientes_letras = "(?P<numero>mono|un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) ?(ambientes|ambiente|amb)"
title_ambientes_numero = "(?P<numero>\d\d?) ?(ambientes|ambiente|amb)"
title_ambientes_letras = "(?P<numero>mono|un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) ?(ambientes|ambiente|amb)"

#Ejecuto la funcion tanto en la columna "decription" como la columna "title"
buscador_patrones(description_ambientes_numero, "description", "numero_ambientes_description")
buscador_patrones(description_ambientes_letras, "description", "numero_ambientes_desc_letras")
buscador_patrones(title_ambientes_numero, "title", "numero_ambientes_title")
buscador_patrones(title_ambientes_letras, "title", "numero_ambientes_title_letras")

In [9]:
data_final.loc[mask_no_rooms,["rooms","description","title","numero_ambientes_description","numero_ambientes_desc_letras","numero_ambientes_title","numero_ambientes_title_letras"]].head(10)

,rooms,description,title,numero_ambientes_description,numero_ambientes_desc_letras,numero_ambientes_title,numero_ambientes_title_letras
0,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,2,NaN,2,NaN
1,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN,NaN,NaN,NaN
2,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,2,NaN,2,NaN
3,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,3,NaN,3,NaN
4,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,NaN,NaN,2,NaN
5,NaN,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",NaN,NaN,NaN,NaN
6,NaN,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",NaN,NaN,NaN,NaN
7,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,NaN,MONO,NaN,MONO
8,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",NaN,DOS,NaN,DOS
9,NaN,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,NaN,NaN,NaN,NaN


In [10]:
#combierto amb_desc_letras y amb_title_letras en minuscula para estandarizar las columnas
data_final["numero_ambientes_desc_letras"] = data_final["numero_ambientes_desc_letras"].str.lower()
data_final["numero_ambientes_title_letras"] = data_final["numero_ambientes_title_letras"].str.lower()

In [11]:
#Reemplazamos aquellos numeros en letras con numeros, lo hacemos con un diccionario
dictionary = {"mono":1, "un":1,"dos":2,"tres":3,"cuatro":4,"cinco":5,"seis":6,"siete":7,"ocho":8,"nueve":9,"diez":10}
data_final["numero_ambientes_desc_letras"] = data_final["numero_ambientes_desc_letras"].replace(dictionary)
data_final["numero_ambientes_title_letras"] = data_final["numero_ambientes_title_letras"].replace(dictionary)

In [12]:
data_final.loc[mask_no_rooms,["numero_ambientes_description","numero_ambientes_desc_letras","numero_ambientes_title","numero_ambientes_title_letras"]].head(10)

,numero_ambientes_description,numero_ambientes_desc_letras,numero_ambientes_title,numero_ambientes_title_letras
0,2,NaN,2,NaN
1,NaN,NaN,NaN,NaN
2,2,NaN,2,NaN
3,3,NaN,3,NaN
4,NaN,NaN,2,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,1.0,NaN,1.0
8,NaN,2.0,NaN,2.0
9,NaN,NaN,NaN,NaN


In [13]:
#Reemplazamos los nulos en rooms con las columnas contruidas
data_final["rooms"].fillna(data_final["numero_ambientes_description"], inplace=True)
data_final["rooms"].fillna(data_final["numero_ambientes_desc_letras"], inplace=True)
data_final["rooms"].fillna(data_final["numero_ambientes_title"], inplace=True)
data_final["rooms"].fillna(data_final["numero_ambientes_title_letras"], inplace=True)

In [14]:
#Vemos como se redujo considerablemente la cantidad de nulos en rooms
data_final["rooms"].isnull().sum()

45394

In [15]:
#Para aquellas propiedades sin la cantidad explicita de ambientes detalle en las columnas description y title vamos a tratar de armarlos con la informacion restante de las mismas.

In [16]:
#Comenzamos con dormitorios

#Patrones
dormitorios_numeros = "(?P<numero>\d\d?) ?(dorm|cuarto)"
dormitorios_letras = "(?P<numero>mono|un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) ?(dorm|cuarto)"

#ejecuto la funcion buscador_patrones
buscador_patrones(dormitorios_numeros, "description", "dormitorios_numeros")
buscador_patrones(dormitorios_letras, "description", "dormitorios_letras")

In [17]:
data_final.loc[:,["dormitorios_numeros","dormitorios_letras"]]

,dormitorios_numeros,dormitorios_letras
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,1,NaN
...,...,...
121215,2,NaN
121216,3,NaN
121217,NaN,NaN
121218,NaN,NaN


In [18]:
#Reemplazo las letras por integers
data_final["dormitorios_letras"] = data_final["dormitorios_letras"].str.lower()
diccionario_letras = {"un":1,"dos":2,"tres":3,"cuatro":4,"cinco":5,"seis":6,"siete":7,"ocho":8,"nueve":9,"diez":10}
data_final["dormitorios_letras"] = data_final["dormitorios_letras"].replace(diccionario_letras)

In [19]:
mascara_nulos = data_final["rooms"].isnull()
data_final.loc[mascara_nulos,["rooms","dormitorios_numeros","dormitorios_letras"]]

,rooms,dormitorios_numeros,dormitorios_letras
1,NaN,NaN,NaN
5,NaN,NaN,2.0
6,NaN,NaN,2.0
9,NaN,2,NaN
10,NaN,3,NaN
...,...,...,...
121212,NaN,1,NaN
121213,NaN,3,NaN
121214,NaN,NaN,NaN
121216,NaN,3,NaN


In [20]:
#Reemplazo los nulos de "dormitorios_numeros" con la info de "dormitorios_letras" para tener todos los dormitorios en una columna
data_final["dormitorios_numeros"].fillna(data_final["dormitorios_letras"], inplace=True)

In [21]:
# Eliminamos los signos de puntuación de la columna description
regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
pattern = re.compile(regex)
data_final["description_clean"] = data_final["description"].apply(lambda x: pattern.sub("", x))
data_final.loc[:,["description_clean","rooms"]]

,description_clean,rooms
0,2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO ...,2
1,Venta de departamento en décimo piso al frente...,NaN
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2
3,PH 3 ambientes con patio Hay 3 deptos en lote ...,3
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,2
...,...,...
121215,TORRE FORUM ALCORTA MÁXIMA CATEGORÍAImpecable...,3
121216,Excelente e impecable casa en Venta en Las Lom...,NaN
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,1
121218,2 Amb al contrafrente luminoso El departament...,2


In [22]:
#Vamos a buscar los siguientes ambientes para poder completar rooms:
#Cocina Comedor
#Living Comedor
#Cocina separada
#Living separado
#Comedor separado

In [23]:
#para no duplicor la busqueda de ambientes voy a reemplazar las palabras (cocina comedor) y (living comedor) en description por palabras claves. Para luego poder buscar por palabras unicas como cocina, living y comedor.
#Cambio (cocina comedor) por la palabra "python"
regex_cocina_comedor = 'cocina comedor'
pattern = re.compile(regex_cocina_comedor)
data_final["description_clean"] = data_final["description"].apply(lambda x: pattern.sub("python", x))

In [24]:
#cambio (living comedor) por "pandas"
regex_living_comedor = 'living comedor'
pattern = re.compile(regex_living_comedor)
data_final["description_clean"] = data_final["description"].apply(lambda x: pattern.sub("pandas", x))

In [25]:
data_final["description_clean"].sample(20)

17109     Venta de Departamento  3 ambientes con depeden...
78290     Venta de Departamento 2 AMBIENTES en Barcelona...
79508     Sergio Villella Inmobiliaria presenta departam...
4971      Casa de 2 ambientes en una planta, sobre lote ...
10937     Venta - Casa - Argentina, Patagonia, Bariloche...
6504      Se vende departamento en Puerto Amarras torre ...
44867     CODIGO: 1084-B092 ubicado en: COLON 2200- MAR ...
49512     Venta de Lote , OSTENDELote unifamiliar, MZ 44...
89128     EXCELENTE CHALET PH AL FRENTEINMUEBLE  CON TRE...
47854     - Toma departamento en parte de pago, zona Oli...
101950    CARACTERÍSTICAS Torre Mirador. Piso exclusivo ...
17898     Venta de Departamento 3 Ambientes con dependen...
27439     Venta o Permuta por Mayor Valor,  Departamento...
9751                    Casa en Venta de 3 dorm. en Cordoba
50011     Venta de Departamento MONOAMBIENTE, Paraná -  ...
85715     Complejo Holmberg de GyG, el dpto. de 2 dormit...
80392     EXCELENTE 2 AMBIENTES CON BALC

In [26]:
#Busco si contienen las siguientes palabras para sumarlas como ambientes
data_final["cocina_comedor"] = data_final.loc[mascara_nulos,"description_clean"].str.contains(r'python?') #que es cocina comedor
data_final["living_comedor"] = data_final.loc[mascara_nulos,"description_clean"].str.contains(r'pandas?') #que es living comedor
data_final["cocina_unica"] = data_final.loc[mascara_nulos, "description_clean"].str.contains(r'cocina?') #cocina unica
data_final["living_unico"] = data_final.loc[mascara_nulos,"description_clean"].str.contains(r'living?') #living unico
data_final["comedor_unico"] = data_final.loc[mascara_nulos,"description_clean"].str.contains(r'comedor?') #comedor unico

In [27]:
data_final.loc[mascara_nulos,["cocina_comedor","living_comedor", "cocina_unica", "living_unico", "comedor_unico"]]

,cocina_comedor,living_comedor,cocina_unica,living_unico,comedor_unico
1,False,False,False,False,True
5,False,False,True,True,True
6,False,False,False,False,False
9,False,False,False,False,False
10,False,True,True,False,False
...,...,...,...,...,...
121212,False,False,True,True,True
121213,False,False,True,True,True
121214,False,False,False,False,False
121216,False,True,True,False,True


In [28]:
#Aquellas propiedas en las que es True les agregamos 1, en aquellas que es False les ponemos 0.
data_final["cocina_comedor"] =  data_final["cocina_comedor"].apply(lambda x: 0 if x == False else 1)
data_final["living_comedor"] =  data_final["living_comedor"].apply(lambda x: 0 if x == False else 1)
data_final["cocina_unica"] =  data_final["cocina_unica"].apply(lambda x: 0 if x == False else 1)
data_final["living_unico"] =  data_final["living_unico"].apply(lambda x: 0 if x == False else 1)
data_final["comedor_unico"] =  data_final["comedor_unico"].apply(lambda x: 0 if x == False else 1)

In [29]:
data_final.loc[mascara_nulos,["cocina_comedor","living_comedor", "cocina_unica", "living_unico", "comedor_unico"]]

,cocina_comedor,living_comedor,cocina_unica,living_unico,comedor_unico
1,0,0,0,0,1
5,0,0,1,1,1
6,0,0,0,0,0
9,0,0,0,0,0
10,0,1,1,0,0
...,...,...,...,...,...
121212,0,0,1,1,1
121213,0,0,1,1,1
121214,0,0,0,0,0
121216,0,1,1,0,1


In [30]:
data_final["dormitorios_numeros"] = data_final["dormitorios_numeros"].astype(float)
data_final["cocina_comedor"] = data_final["cocina_comedor"].astype(float)
data_final["living_comedor"] = data_final["living_comedor"].astype(float)
data_final["cocina_unica"] = data_final["cocina_unica"].astype(float)
data_final["living_unico"] = data_final["living_unico"].astype(float)
data_final["comedor_unico"] = data_final["comedor_unico"].astype(float)

In [31]:
#Sumamos la cantidad de ambientes y lo almacenamos en la columna ambientes
columnas_lista = ["dormitorios_numeros","cocina_comedor", "living_comedor","cocina_unica", "living_unico", "comedor_unico"]
columnas_lista
data_final["ambientes"] = data_final[columnas_lista].sum(axis=1)
data_final.loc[mascara_nulos,["rooms","dormitorios_numeros","cocina_comedor","living_comedor","cocina_unica","living_unico","comedor_unico","ambientes"]].sample(15)

,rooms,dormitorios_numeros,cocina_comedor,living_comedor,cocina_unica,living_unico,comedor_unico,ambientes
104936,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0
4519,NaN,2.0,0.0,0.0,1.0,1.0,1.0,5.0
73307,NaN,2.0,0.0,0.0,1.0,0.0,1.0,4.0
73746,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0
16397,NaN,5.0,0.0,0.0,1.0,1.0,1.0,8.0
116114,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0
76022,NaN,4.0,0.0,0.0,0.0,0.0,0.0,4.0
105606,NaN,NaN,0.0,0.0,1.0,0.0,1.0,2.0
86234,NaN,1.0,0.0,1.0,1.0,0.0,0.0,3.0
48282,NaN,2.0,0.0,0.0,1.0,1.0,1.0,5.0


In [32]:
#Para aquellas propiedades que tenemos la cantidad de dormitorios y no tienen cocina_comedor ni cocina les agregamos 1 ambiente que seria la cocina. 
data_final["ambientes"] = data_final.apply(lambda x: x.ambientes + 1 if (x.dormitorios_numeros > 0) and (x.cocina_comedor == 0) and (x.cocina_unica == 0) else x.ambientes, axis=1)

In [33]:
data_final.loc[mascara_nulos,["rooms","dormitorios_numeros","cocina_comedor","living_comedor","cocina_unica","living_unico","comedor_unico","ambientes"]].head(10)

,rooms,dormitorios_numeros,cocina_comedor,living_comedor,cocina_unica,living_unico,comedor_unico,ambientes
1,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0
5,NaN,2.0,0.0,0.0,1.0,1.0,1.0,5.0
6,NaN,2.0,0.0,0.0,0.0,0.0,0.0,3.0
9,NaN,2.0,0.0,0.0,0.0,0.0,0.0,3.0
10,NaN,3.0,0.0,1.0,1.0,0.0,0.0,5.0
15,NaN,3.0,0.0,1.0,1.0,0.0,0.0,5.0
18,NaN,3.0,0.0,0.0,0.0,0.0,1.0,5.0
20,NaN,4.0,0.0,0.0,1.0,0.0,0.0,5.0
22,NaN,3.0,0.0,0.0,1.0,1.0,0.0,5.0
26,NaN,2.0,0.0,1.0,1.0,0.0,1.0,5.0


In [34]:
data_final["ambientes"].unique()

array([  5.,   1.,   6.,   3.,   7.,  35.,   8.,   0.,  10.,   4.,   9.,
        59.,   2.,  27.,  13.,  28.,  11.,  50.,  40.,  63.,  98.,  86.,
        23.,  45.,  39.,  85.,  89.,  37.,  33.,  55.,  75.,  30.,  80.,
        65.,  12.,  83.,  15.,  25.,  46.,  34.,  26.,  16.,  96.,  76.,
        36.,  77.,  66.,  47.,  57.,  17.,  88.,  87.,  67.,  18.,  14.,
        95.,  82.,  54.,  51.,  78.,  53.,  90.,  19.,  48.,  24.,  62.,
       103.,  79.,  70.,  49.,  68.,  20.,  56.,  42.,  81.,  43.,  92.,
        61.,  38.,  58.,  29.,  52.,  22.,  91.,  32.])

In [35]:
data_final["ambientes"] = data_final["ambientes"].replace(0, np.nan)
data_final.loc[mascara_nulos,["rooms","dormitorios_numeros","cocina_comedor","living_comedor","cocina_unica","living_unico","comedor_unico","ambientes"]].head(20)

,rooms,dormitorios_numeros,cocina_comedor,living_comedor,cocina_unica,living_unico,comedor_unico,ambientes
1,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0
5,NaN,2.0,0.0,0.0,1.0,1.0,1.0,5.0
6,NaN,2.0,0.0,0.0,0.0,0.0,0.0,3.0
9,NaN,2.0,0.0,0.0,0.0,0.0,0.0,3.0
10,NaN,3.0,0.0,1.0,1.0,0.0,0.0,5.0
15,NaN,3.0,0.0,1.0,1.0,0.0,0.0,5.0
18,NaN,3.0,0.0,0.0,0.0,0.0,1.0,5.0
20,NaN,4.0,0.0,0.0,1.0,0.0,0.0,5.0
22,NaN,3.0,0.0,0.0,1.0,1.0,0.0,5.0
26,NaN,2.0,0.0,1.0,1.0,0.0,1.0,5.0


In [36]:
#reemplazamos los nulos en rooms con la cantidad de ambientes que construimos
data_final["rooms"].fillna(data_final["ambientes"], inplace=True)
data_final["rooms"].isnull().sum()

12164

In [39]:
#eliminamos aquellas columnas que no necesitamos mas
data_final.drop(columns=["numero_ambientes_title", "numero_ambientes_desc_letras", "numero_ambientes_description", "description_clean","cocina_comedor","living_comedor","cocina_unica","living_unico","comedor_unico","dormitorios_letras","dormitorios_numeros","dormitorios_numeros","ambientes","numero_ambientes_title_letras"], inplace=True)

KeyError: "['numero_ambientes_title' 'numero_ambientes_desc_letras'\n 'numero_ambientes_description' 'description_clean' 'cocina_comedor'\n 'living_comedor' 'cocina_unica' 'living_unico' 'comedor_unico'\n 'dormitorios_letras' 'dormitorios_numeros' 'dormitorios_numeros'\n 'ambientes'] not found in axis"

In [38]:
data_final.head()

,property_type,place_name,place_with_parent_names,state_name,price,currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_per_m2,rooms,description,title,numero_ambientes_title_letras
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Capital Federal,62000.0,USD,62000.0,55.0,40.0,1550.000000,2,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,NaN
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Bs.As. G.B.A. Zona Sur,150000.0,USD,150000.0,NaN,NaN,NaN,1,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Capital Federal,72000.0,USD,72000.0,55.0,55.0,1309.090909,2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,NaN
3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Capital Federal,95000.0,USD,95000.0,NaN,NaN,NaN,3,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,NaN
4,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Buenos Aires Costa Atlántica,64000.0,USD,64000.0,35.0,35.0,1828.571429,2,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,NaN


In [40]:
#Vemos que la cantidad de nulos en rooms bajo considerablemente.
data_final["rooms"].isnull().sum()

12164